In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents1.json').read()
intents = json.loads(data_file)

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [ ]:

# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

616 documents
44 classes ['Causes of heart disease', 'Effect of diet on heart disease', 'Effect of medical condition of patient on heart disease', 'Effect of physical activity on heart diease', 'Error', 'Heart Disease', 'Impact of exercise on heart disease', 'Impact of heart disease', 'Response not correct', 'Role of Age in heart disease', 'Role of Smoking in heart disease', 'Role of blood pressure in heart disease', 'Types of heart disease', 'appointment with heart specialist', 'causes of diabetes', 'causes of pneumonia', 'chest pain related to heart disease', 'diagnosis of heart disease', 'effect of cholesterol on heart disease', 'effect of fasting blood sugar on heart disease', 'emergency situations in heart disease', 'factors inducing heart disease', 'greeting', 'health_risks_obesity', 'heart rate', 'importance of exercise for diabetes', 'introduction', 'introduction to diabetes', 'introduction to pneumonia', 'medications for heart disease', 'monitoring of heart health', 'personal'

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in an attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in the current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '0' for each tag and '1' for the current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features
random.shuffle(training)

# Extract features and labels from the training list
features = [x[0] for x in training]
labels = [x[1] for x in training]

# Pad sequences to make them uniform in length
max_length = max(len(seq) for seq in features)
padded_features = pad_sequences(features, maxlen=max_length, padding='post')

# Convert features and labels to NumPy arrays
train_x = np.array(padded_features)
train_y = np.array(labels)

print("Training data created")

Training data created


In [ ]:


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons, and 3rd output layer contains the number of neurons equal to the number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.002 , momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("Model created")


Epoch 1/100
124/124 [==============================] - 1s 3ms/step - loss: 3.7828 - accuracy: 0.0325
Epoch 2/100
124/124 [==============================] - 0s 2ms/step - loss: 3.7249 - accuracy: 0.0649
Epoch 3/100
124/124 [==============================] - 0s 2ms/step - loss: 3.6436 - accuracy: 0.0779
Epoch 4/100
124/124 [==============================] - 0s 2ms/step - loss: 3.5540 - accuracy: 0.0860
Epoch 5/100
124/124 [==============================] - 0s 2ms/step - loss: 3.4893 - accuracy: 0.0909
Epoch 6/100
124/124 [==============================] - 0s 2ms/step - loss: 3.4206 - accuracy: 0.0990
Epoch 7/100
124/124 [==============================] - 0s 2ms/step - loss: 3.2858 - accuracy: 0.1282
Epoch 8/100
124/124 [==============================] - 0s 2ms/step - loss: 3.1814 - accuracy: 0.1834
Epoch 9/100
124/124 [==============================] - 0s 2ms/step - loss: 3.0990 - accuracy: 0.1672
Epoch 10/100
124/124 [==============================] - 0s 2ms/step - loss: 2.9404 - accura

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:

import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from tensorflow.keras.models import load_model


nltk.download('punkt')
nltk.download('wordnet')


model = load_model('chatbot_model.h5')
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
lemmatizer = WordNetLemmatizer()

def clean_up_sentence(sentence):
    # Tokenize the user input and lemmatize each word
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):

    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):

    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result


print("Chatbot Testing Interface")
print("Type 'quit' to exit")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break


    ints = predict_class(user_input, model)
    res = get_response(ints, intents)
    print("Bot:", res)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Chatbot Testing Interface
Type 'quit' to exit
You: hi
1/1 [==============================] - 0s 90ms/step
Bot: Hi there! Welcome to Health Bot. How can I assist you today?
You: what si diabeties
1/1 [==============================] - 0s 22ms/step
Bot: Hi there, how can I help?
You: what is diabeties
1/1 [==============================] - 0s 25ms/step
Bot: Mitral valve prolapse is a condition where the valve between the left upper and lower chambers of the heart doesn't close properly.
You: diabetes
1/1 [==============================] - 0s 31ms/step
Bot: While there is currently no cure for diabetes, effective management can help control blood sugar levels and prevent or delay complications.
You: hmm
1/1 [==============================] - 0s 20ms/step
Bot: I'm sorry, but I can only provide assistance with questions related to Heart Disease inquiries. If you have any questions about our services, please don't hesitate to ask, and I'll be glad to help!
You: hello
1/1 [===================

KeyboardInterrupt: Interrupted by user